# Write-up on 3D Motion Planning Report


## Project Scope
This project is about to develop an algorithm that is able to plan a path from a start to a goal location trough an urban environment for a quadcopter.

The algorithm is developed and run within the synthetic environment  provided by the Udacity drone simulator. (you can download the simulator [here](https://github.com/udacity/FCND-Simulator-Releases/releases)).


## Writeup
This brief report discuss on how the [rubric points](https://review.udacity.com/#!/rubrics/1534/view) for passing the project were addressed. It includes brief explanation, supporting images that explain how each rubric item was addressed and in particula, the code in which each step is handled.


## Explain the Starter Code

### Explain the functionality of what's provided in `motion_planning.py` and `planning_utils.py`

`motion_planning.py` is a modified version of `backyard_flyer_solution.py`, that implements a code for commanding a quadcopter. The code that controls the drone is written in python following a [event-driven programming paradigm](https://en.wikipedia.org/wiki/Event-driven_programming). This allow us to write programs that respond dynamically to a changing and unpredictable environment.

The main difference between both files is that the flying path is hard-coded in `backyard_flyer_solution.py` to flight following a simple square, whereas `motion_planning.py` includes a PLANNING state, in which a planner evaluates a feasible flighig path from the given start and goal locations. The code of the planner is launched from the `plan_path()` method and its building blocks are placed in the `planning_utils.py`. This starter planner uses an A* algorithm for planning based on an grid obstacle representation and Euclidean distance heuristic.

As required, both `backyard_flyer_solution.py` and `motion_planning.py` were successfully tested before starting the implementation of my own path planning algorithm.

The `planning_utils.py` file contains the code for creating a grid from a 2.5D map representation, for defining the action space of the drone and that for the A* algorithm, included its heuristic method. However, it did not include any diagonal movement in the action space. Therefore, the planner algorithm as is at the starter code is not able to offer the drone any other path but a zigzag series of waypoints when it is asked to get a goal located somewhere in a diagonal location regarding the start location. And this is exactly the situation implemented in the starter code (see image below).

![ZigZag movement](./misc/Zigzag_with_starter_code.png)


## Implementing Your Path Planning Algorithm

### 1. Set your global home position
Home position of the drone is set to be placed to any arbitrary coordinates given in global frame (latitude, longitude, altitude). In particular, in this implementation, the global home position is placed at the center of the map provided, corresponding to San Francisco downtown. Latitude and longitude coordinates of this point are given in the first line of the `colliders.csv` file.

Next code in `motion_planning.py` shows the extraction of the center coordinates of the map and how those are set to the global home position of the drone. Note that altitude is set to ground level (0 m).

![Setting Global Home Position](./misc/setting_drone_home_position.png)
.
 
Method `extract_map_centre()` is placed at `planning_utils.py` as follows:

![Method for extracting map center](./misc/extract_map_center.png)
.

### 2. Set your current local position
The current local position of the drone can be obtained using `global_position`, a `Drone` class attribute . It gives the current GPS position of the drone.

Then, the current position of the drone in global frame is turned into local frame (NED frame centered in the global home position of the drone) by means of the `global_to_local()` method, which is called from `planning_utils.py`.

![Setting current local position](./misc/setting_current_local_position.png)
.


### 3. Set grid start position from local position
Once the current position of the drown before taking-off is knwon, it makes sence to appoint it as the start location of the drone. This is done by directly passing the local position of the drone. Note the sign change in the altitude value.

![Defining start location](./misc/defining_start_position.png)
.


### 4. Set grid goal position from geodetic coords
Goal location in global frame is passed in the code from the input values for latitude, longitude and minimum flight altitude given by the user when running `motion_planning.py`. This is done by means of the use of the [argparse module](https://docs.python.org/3/library/argparse.html). Default values are passed in, if the user does not specify any desired goal location. 

![Defining goal location](./misc/goal_location_argparse.png)
.


Then, following the same procedure than with the definition of the start location, the goal location in global frame is turned into local frame by means of the `global_to_local()` method and the result is appointed as the goal location, after having changed the sign change in the altitude value.


![Defining goal location](./misc/defining_goal_position.png)
.


### 5. Modify A* to include diagonal motion (or replace A* altogether)
The planner is built on the top of A* algorithm properly adapted to deal with graphs. The graph is created following Probabilistic Roadmap, that allows us to build a 3D representation of the planning problem.

Following the Probabilistic Roadmap procedure, a number of nodes are sampled over the volume bounded by the map plan and the minimum and maximum flight altitude specified by the user when launching the planner. Sampled nodes are then checked for collision with obstacles and discarded when appropriate. At last, a graph joining the remainder nodes through free space is created.

All this is done with `create_probabilistic_roadmap()` method at `planning_utils.py`. The method makes use of [Shapely geometry classes](https://shapely.readthedocs.io/en/stable/geometry.html) to turn obstacles into polygons, to deal with the determination of which nodes do collide with obstacles and to find edges between nodes running through free space. The method also makes use of [NetworkX](https://networkx.org) and [KDTree]() instances for creating the graph.

![create_probabilistic_roadmap() method](./misc/create_PRM.png)
.



Note that the creation of the graph has been placed in the `__main_` code of the `motion_planning.py` and then it is passed in, together with other parameters, to the `MotionPlanning` class. It was observed that placing the code this way is computationally much better than creating the graph once the connection of the drone is established.

![Graph creation at `__main__`](./misc/graph_creation_at_main.png)
.


Finally, A* algorithm, with Euclidean distance heuristic, is run over the graph to get a feasible path from start to goal location. Note that since is is quite unlikely that any of the sampled nodes that are now part of the graph match with start or goal locations, the A* is run with the closest nodes of the graph to start and goal locations instead. And later, the actual start and goal locations are added to the path as the first and last position respectively.

![Planning the past with A* from graph](./misc/path_form_A_star.png)
.

At the end the path, after having been pruned, is converted to waypoints. These are returned in local frame coordinates (plus drone heading).


![Conversion to waypoints](./misc/conversion_to_waypoints.png)
.


#### 6. Cull waypoints 
Unnecessary waypoints within the feasible path provided by the A* algorithm are pruned. To that end, ray tracing method is implemented in `path_prunning()` method, which take advantage of [Shapely geometry classes](https://shapely.readthedocs.io/en/stable/geometry.html) to carry out this task.

![Path pruning](./misc/path_pruning_method.png)
.



## Execute the flight
### 1. Does it work?
The planner implemented in `motion_planning.py` has been tested with different start and goal locations successfully in terms of computational time and proper path definition. Here below you can find an example of flight planning: orange line is the original path from A* and blue one is the pruned path.


![Flight_Planning](./misc/Flight_planning_PRM.png)
.